# Set up a WQ-MAKER run using the Jupyter Notebook

## Step 1. Get oriented. 

You will find staged example data in "/opt/WQ-MAKER_example_data/" within the MASTER instance. List its contents with the `ls` command:

In [1]:
!ls /opt/WQ-MAKER_example_data/

maker_bopts.ctl  maker_opts.ctl     worker-launch.yml.BU
maker_exe.ctl	 test_data	    WQ-MAKER-Jupyter-notebook.ipynb
maker-hosts	 worker-launch.yml


In [2]:
!ls /opt/WQ-MAKER_example_data/test_data

mRNA.fasta		  Os-rRNA.fa	       test_genome_chr1.fasta
msu-irgsp-proteins.fasta  plant_repeats.fasta  test_genome.fasta


- maker_*.ctl file are a set of configuration files that can be used for this exercise or generated as described below. 
- worker-launch.yml and maker-hosts are ansible-playbook and host file for luanching jobs on WORKERS (optional for WQ-MAKER)
- fasta files include a scaled-down genome (test_genome.fasta) which is comprised of the first 300kb of 12 chromosomes of rice and scaled-down genome (test_genome_chr1.fasta) which is comprised of the first 300kb of first chromosome of rice
- mRNA sequences from NCBI (mRNA.fasta)
- publicly available annotated protein sequences of rice (MSU7.0 and IRGSP1.0) - msu-irgsp-proteins.fasta
collection of plant repeats (plant_repeats.fasta)
- ribosomal RNAsequence of rice (Os-rRNA.fa)

Executables for running MAKER are located in /opt/maker/bin and /opt/maker/exe:

In [3]:
!ls /opt/maker/bin/

cegma2zff	   gff3_merge	   maker2wap		   map_fasta_ids
chado2gff3	   iprscan2gff3    maker2zff		   map_gff_ids
compare		   iprscan_wrap    maker_functional	   mpi_evaluator
cufflinks2gff3	   ipr_update_gff  maker_functional_fasta  mpi_iprscan
evaluator	   maker	   maker_functional_gff    tophat2gff3
fasta_merge	   maker2chado	   maker_map_ids
fasta_tool	   maker2eval_gtf  map2assembly
genemark_gtf2gff3  maker2jbrowse   map_data_ids


As the names suggest the **/opt/maker/bin** directory includes many useful auxiliary scripts.  For example cufflinks2gff3 will convert output from an RNA-seq analysis into a GFF3 file that can be used for input as evidence for WQ-MAKER. RepeatMasker, augustus, blast, exonerate, and snap are programs that MAKER uses in its pipeline.  We recommend reading [MAKER Tutorial](http://weatherby.genetics.utah.edu/MAKER/wiki/index.php/MAKER_Tutorial_for_GMOD_Online_Training_2014) at GMOD for more information about these.

### Step 2. Set up a WQ-MAKER run.  Create a working directory called "maker_run" on your home directory using the mkdir command and use cd to move into that directory:

Navigate to the mounted volume for creating test directory

In [4]:
%cd /vol_b

/vol_b


In [5]:
!mkdir wq_maker_run
%cd wq_maker_run

/vol_b/wq_maker_run


### Step 3. Copy the contents of  "WQ-MAKER_example_data" into the current directory using cp -r command.  Verify using the ls command.  Change the permissions on that directory

In [6]:
!cp -r /opt/WQ-MAKER_example_data/test_data .

In [7]:
!maker -h

Argument "2.53_01" isn't numeric in numeric ge (>=) at /usr/local/lib/x86_64-linux-gnu/perl/5.22.1/forks.pm line 1570.

MAKER version 2.31.9

Usage:

     maker [options] <maker_opts> <maker_bopts> <maker_exe>


Description:

     MAKER is a program that produces gene annotations in GFF3 format using
     evidence such as EST alignments and protein homology. MAKER can be used to
     produce gene annotations for new genomes as well as update annotations
     from existing genome databases.

     The three input arguments are control files that specify how MAKER should
     behave. All options for MAKER should be set in the control files, but a
     few can also be set on the command line. Command line options provide a
     convenient machanism to override commonly altered control file values.
     MAKER will automatically search for the control files in the current
     working directory if they are not specified on the command line.

     Input files listed in the control options fil

### Step 5.  Create control files that tell MAKER what to do. Three files are required:

- maker_opts.ctl - gives location of input files (genome and evidence) and sets options that affect MAKER behavior
- maker_exe.ctl - gives path information for the underlying executables.
- maker_bopt.ctl - sets parameters for filtering BLAST and Exonerate alignment results

In [8]:
!maker -CTL
!ls

Argument "2.53_01" isn't numeric in numeric ge (>=) at /usr/local/lib/x86_64-linux-gnu/perl/5.22.1/forks.pm line 1570.
maker_bopts.ctl  maker_exe.ctl	maker_opts.ctl	test_data


- The "maker_exe.ctl" is automatically generated with the correct paths to executables and does not need to be modified.  
- The "maker_bopt.ctl" is automatically generated with reasonable default parameters and also does not need to be modified unless you want to experiment with optimization of these parameters.
- The automatically generated "maker_opts.ctl" file **needs to be modified** in order to specify the genome file and evidence files to be used as input.  You can use the text editor **"vi"** or **"nano"** that is already installed in the **MASTER** instance

In [9]:
# If pressed for time a pre-edited version of the "maker_opts.ctl" file is staged in /opt/WQ-MAKER_example_data. Delete the current file and copy the staged version here and then skip to **Step 6**.
!rm maker_opts.ctl
!cp /opt/WQ-MAKER_example_data/maker_opts.ctl .

**Otherwise** open the maker_opts.ctl in a text editor of choice (**vi** or **nano**) on the command line and edit the following changes in the **maker_opts.ctl** file. Here are the sections of the "maker_opts.ctl" file you need to edit.  For more information about the this please check this [The_MAKER_control_files_explained](http://weatherby.genetics.utah.edu/MAKER/wiki/index.php/The_MAKER_control_files_explained) - Add path information to files as shown.

* Do not allow any spaces after the equal sign or anywhere else.
* The files can be present in same the directory as the "maker_opts.ctl" or make sure you use the relative path if the files are present in other directories

This section pertains to specifying the genome assembly to be annotated and setting organism type:

#-----Genome (these are always required)

genome=./test_data/test_genome.fasta #genome sequence (fasta file or fasta embeded in GFF3 file)

The following section pertains to EST and other mRNA expression evidence.  Here we are only using same species data, but one could specify data from a related species using the "altest" parameter. With RNA-seq data aligned to your genome by Cufflinks or Tophat one could use maker auxiliary scripts (cufflinks2gff3 and tophat2gff3) to generate GFF3 files and specify these using the est_gff parameter:

#-----EST Evidence (for best results provide a file for at least one)

est=./test_data/mRNA.fasta #set of ESTs or assembled mRNA-seq in fasta format

altest= #EST/cDNA sequence file in fasta format from an alternate organism

The following section pertains to protein sequence evidence.  Here we are using previously annotated protein sequences.  Another option would be to use SwissProt or other database:

#-----Protein Homology Evidence (for best results provide a file for at least one)

protein=./test_data/msu-irgsp-proteins.fasta  #protein sequence file in fasta format (i.e. from mutiple oransisms)

This next section pertains to repeat identification:

#-----Repeat Masking (leave values blank to skip repeat masking)

model_org= #select a model organism for RepBase masking in RepeatMasker

rmlib=./test_data/plant_repeats.fasta #provide an organism specific repeat library in fasta format for RepeatMasker

### Step 6.  Run WQ-MAKER on MASTER

In [12]:
import os
os.system("wq_maker -contigs-per-split 1 -cores 1 -memory 2048 -disk 4096 -N wq_test_${USER} -d all -o master.dbg -debug_size_limit=0 > log_file 2>&1 &")

0

* -contigs-per-split 1: splits the genome file into 1 contig/scaffold/sequence per file. By specifiying this option, we are telling wq_maker to split the genome file into 1 sequence per file. By default, the wq_maker splits the fasta file into 10 sequences per file and this case, it is not ideal because, there will be 2 files (1 containing chromosomes from 1-10 and the other containing 11-12). This will decrease the speed at the wq_maker annotates the genome. 

**Unless otherwise you have a complete genome containing chromosomes or very few scaffolds, it is not recommended to use this option. For example if you have a genome that contains 10,000 sequences, then this option will create 10,000 files on your working directory which is not ideal of navigation purposes. Check to see how many contigs/scaffolds/chromosomes you have in your genome using 
$$grep ">" -c <genome fasta file>$$
and if the number is too high, then avoid this option**

* -N maker_run_ud sets the project name to wq_test_{USER}. This is mandatory if we need to run WQ-MAKER. 
* -d all Sets the debug flag for Work Queue. For all debugging output, try 'all'
* -o master.dbg Sets the debug file for Work Queue
* -debug_size_limit=0 Sets the byte wrap around on the debug file. 0 signifies it is never wrapped (Default it 1M)
* -stats test_out_stats.txt Specifies the file were Work Queue master stats are written
* log_file.txt captures the stdout

It takes 5-10 minutes (depending on the genome size) for the MASTER to advertise master status to the catalog server before your run WQ-MAKER on the WORKERS. To check if the MASTER has advertised master status to the catalog server you can run

In [13]:
!tail log_file

Fri Oct 20 17:27:14 2017 :: Submitting file ./test_data/test_genome.fasta_000003 for processing.
Fri Oct 20 17:27:14 2017 :: Submitted task 9 for annotating ./test_data/test_genome.fasta_000003 with command:  maker -g ./test_data/test_genome.fasta_000003 -base test_genome -debug_size_limit=0
Fri Oct 20 17:27:14 2017 :: Submitting file ./test_data/test_genome.fasta_000004 for processing.
Fri Oct 20 17:27:14 2017 :: Submitted task 10 for annotating ./test_data/test_genome.fasta_000004 with command:  maker -g ./test_data/test_genome.fasta_000004 -base test_genome -debug_size_limit=0
Fri Oct 20 17:27:14 2017 :: Submitting file ./test_data/test_genome.fasta_000001 for processing.
Fri Oct 20 17:27:14 2017 :: Submitted task 11 for annotating ./test_data/test_genome.fasta_000001 with command:  maker -g ./test_data/test_genome.fasta_000001 -base test_genome -debug_size_limit=0
Fri Oct 20 17:27:14 2017 :: Submitting file ./test_data/test_genome.fasta_000011 for processing.
Fri Oct 20 17:27:14 20

When you see these two lines you can start running jobs on WORKERS

`warning: this work queue master is visible to the public.
warning: you should set a password with the --password option.`

### Step 7.  Run WQ-MAKER on WORKERS

For running WQ-MAKER on WORKERS you need three files. 
* Ansible config file
* Maker hostes file
* Ansible playbook

\1. Copy *ansible.cfg* file into your home directory which will help you to avoid host verification

In [14]:
!cp /opt/WQ-MAKER_example_data/.ansible.cfg ~

In [15]:
!cat ~/.ansible.cfg

[defaults]
host_key_checking = False


\2. Copy `maker-hosts` file into your working directory and populate it with ip addresses of the workers

In [16]:
!cp /opt/WQ-MAKER_example_data/maker-hosts .
!echo "149.165.169.21" >> maker-hosts # This ip address is specific to my account. This will not work for you
!echo "149.165.169.78" >> maker-hosts # This ip address is specific to my account. This will not work for you

In [17]:
!cat maker-hosts

[workers]
149.165.169.21
149.165.169.78


\3.Copy the Ansible playbook to your working directory

In [18]:
!cp /opt/WQ-MAKER_example_data/worker-launch.yml .

In [19]:
!cat worker-launch.yml

---
- hosts : workers
  environment:
    PATH: "{{ ansible_env.PATH }}:/home/${USER}/bin:/home/${USER}/.local/bin:/opt/icommands:/opt/icommands:/opt/exonerate-2.2.0-x86_64/bin/:/opt/cctools/bin:/opt/ncbi-blast-2.6.0+/bin/:/opt/snoscan-0.9.1/:/opt/tRNAscan-SE-1.3.1/:/opt/snap/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/opt/augustus-3.2.2/bin:/opt/maker/bin:/opt/RepeatMasker:/opt/snap"
    PERL5LIB: "/opt/tRNAscan-SE-1.3.1::/opt/cctools/lib/perl5/site_perl"
  tasks :
  - name : Execute the script
    shell : /opt/cctools/bin/work_queue_worker -N wq_test_${USER} -s /home/${USER} --cores all --debug-rotate-max=0 -d all -o /home/${USER}/worker.dbg


* -hosts is the name of the hosts (workers in this case. It can be anything)
tasks is the task that need to be performed by the Ansible (In this case run work_queue_worker)
name is just name of the task (It can be anything) 
* -N maker_run_test sets the project name to maker_run_test. This is mandatory if we need to run WQ-MAKER
* -s /home/upendra/ Set the location for creating the working directory of the worker
* --debug-rotate-max=0 Set the maximum size of the debug log (default 10M, 0 disables)
* -d all Sets the debug flag for Work Queue. For all debugging output, try 'all'
* -o worker.dbg Sets the debug file for Work Queue

Run WQ-MAKER on the WORKERS now

In [20]:
os.system("ansible-playbook -u ${USER} -i maker-hosts worker-launch.yml > log_file_2.txt 2>&1 &")

0

After 1-2 minutes check the status of the WQ-MAKER job, using the following command. 

In [21]:
!work_queue_status -M wq_test_${USER}

PROJECT            HOST                   PORT WAITING RUNNING COMPLETE WORKERS 
wq_test_upendra    js-129-114-104-169.je  9155       0      12        0       2 


### Step 8. Stats output from MASTER instance 

In [22]:
!cat log_file

Argument "2.53_01" isn't numeric in numeric ge (>=) at /usr/local/lib/x86_64-linux-gnu/perl/5.22.1/forks.pm line 1570.
WQ_MAKER supports version 2.31.8. Current version : 2.31.9
Argument "2.53_01" isn't numeric in numeric ge (>=) at /usr/local/lib/x86_64-linux-gnu/perl/5.22.1/forks.pm line 1570.
Genome file not specified. Using the specification in the CTL files.
Argument "2.53_01" isn't numeric in numeric ge (>=) at /usr/local/lib/x86_64-linux-gnu/perl/5.22.1/forks.pm line 1570.
Unknown option: debug_size_limit
STATUS: Parsing control files...
STATUS: Processing and indexing input FASTA files...
Contigs per split : 1
Contigs found : 12
Total number of input files : 12
Argument "2.53_01" isn't numeric in numeric ge (>=) at /usr/local/lib/x86_64-linux-gnu/perl/5.22.1/forks.pm line 1570.
STATUS: Parsing control files...
STATUS: Processing and indexing input FASTA files...
STATUS: Setting up database for any GFF3 input...
A data structure will be created for you at:
/vol_b/wq_maker_run/te

The following are the output files from WQ-MAKER 

In [23]:
!ls test_genome.maker.output

maker_bopts.log  maker_opts.log  test_genome_datastore
maker_exe.log	 mpi_blastdb	 test_genome_master_datastore_index.log


* The maker_opts.log, maker_exe.log, and maker_bopts.log files are logs of the control files used for this run of MAKER.
* The mpi_blastdb directory contains FASTA indexes and BLAST database files created from the input EST, protein, and repeat databases.
* test_genome_master_datastore_index.log contains information on both the run status of individual contigs and information on where individual contig data is stored.
* The test_genome_datastore directory contains a set of subfolders, each containing the final MAKER output for individual contigs from the genomic fasta file.

Check the test_genome_master_datastore_index.log and task_outputs.txt to see if there were any failures:

In [24]:
!cat test_genome.maker.output/test_genome_master_datastore_index.log

Chr1	test_genome_datastore/41/30/Chr1/	STARTED
Chr10	test_genome_datastore/7C/72/Chr10/	STARTED
Chr11	test_genome_datastore/1E/AA/Chr11/	STARTED
Chr12	test_genome_datastore/1B/FA/Chr12/	STARTED
Chr2	test_genome_datastore/E9/36/Chr2/	STARTED
Chr3	test_genome_datastore/CC/EF/Chr3/	STARTED
Chr4	test_genome_datastore/A3/11/Chr4/	STARTED
Chr5	test_genome_datastore/8A/9B/Chr5/	STARTED
Chr6	test_genome_datastore/13/44/Chr6/	STARTED
Chr7	test_genome_datastore/91/B7/Chr7/	STARTED
Chr8	test_genome_datastore/9A/9E/Chr8/	STARTED
Chr9	test_genome_datastore/87/90/Chr9/	STARTED
Chr1	test_genome_datastore/41/30/Chr1/	FINISHED
Chr10	test_genome_datastore/7C/72/Chr10/	FINISHED
Chr11	test_genome_datastore/1E/AA/Chr11/	FINISHED
Chr12	test_genome_datastore/1B/FA/Chr12/	FINISHED
Chr2	test_genome_datastore/E9/36/Chr2/	FINISHED
Chr3	test_genome_datastore/CC/EF/Chr3/	FINISHED
Chr4	test_genome_datastore/A3/11/Chr4/	FINISHED
Chr5	test_genome_datastore/8A/9B/Chr5/	FINISHED
Chr6	test_genome_datastore/13/44/Chr6/	F

All completed.  Other possible status entries include:

**FAILED** - indicates a failed run on this contig, MAKER will retry these

**RETRY** - indicates that MAKER is retrying a contig that failed

**SKIPPED_SMALL** - indicates the contig was too short to annotate (minimum contig length is specified in maker_opt.ctl)

**DIED_SKIPPED_PERMANENT** - indicates a failed contig that MAKER will not attempt to retry (number of times to retry a contig is specified in maker_opt.ctl)

The actual output data is stored in in nested set of directories under* test_genome_datastore* in a nested directory structure. A typical set of outputs for a contig looks like this:

In [25]:
!ls test_genome.maker.output/test_genome_datastore/*/*/*

test_genome.maker.output/test_genome_datastore/13/44/Chr6:
Chr6.gff		   Chr6.maker.snoscan.transcripts.fasta  run.log
Chr6.maker.proteins.fasta  Chr6.maker.transcripts.fasta		 theVoid.Chr6

test_genome.maker.output/test_genome_datastore/1B/FA/Chr12:
Chr12.gff			       Chr12.maker.transcripts.fasta
Chr12.maker.proteins.fasta	       run.log
Chr12.maker.snoscan.transcripts.fasta  theVoid.Chr12

test_genome.maker.output/test_genome_datastore/1E/AA/Chr11:
Chr11.gff			       Chr11.maker.transcripts.fasta
Chr11.maker.proteins.fasta	       run.log
Chr11.maker.snoscan.transcripts.fasta  theVoid.Chr11

test_genome.maker.output/test_genome_datastore/41/30/Chr1:
Chr1.gff		   Chr1.maker.snoscan.transcripts.fasta  run.log
Chr1.maker.proteins.fasta  Chr1.maker.transcripts.fasta		 theVoid.Chr1

test_genome.maker.output/test_genome_datastore/7C/72/Chr10:
Chr10.gff			       Chr10.maker.transcripts.fasta
Chr10.maker.proteins.fasta	       run.log
Chr10.maker.snoscan.transcripts.fasta  theVoid.Chr10

test_

* The Chr6.gff file is in GFF3 format and contains the maker gene models and underlying evidence such as repeat regions, alignment data, and ab initio gene predictions, as well as fasta sequence.  Having all of these data in one file is important to enable visualization of the called gene models and underlying evidence, especially using tools like Apollo which enable manual editing and curation of gene models.
* The fasta files Chr6.maker.proteins.fasta and Chr6.maker.transcripts.fasta contain the protein and transcript sequences for the final MAKER gene calls.
* The Chr6.maker.non_overlapping_ab_initio.proteins.fasta and Chr6.maker.non_overlapping_ab_initio.transcripts.fasta files are models that don't overlap MAKER genes that were rejected for lack of support.
* The Chr6.maker.snap_masked.proteins.fasta and Chr6.maker.snap_masked.transcript.fasta are the initial SNAP predicted models not further processed by MAKER

The output directory **theVoid.Chr1** contains raw output data from all of the pipeline steps. One useful file found here is the repeat-masked version of the contig, query.masked.fasta.

### Step 9: Merge the gff files

In [26]:
%cd test_genome.maker.output/
!gff3_merge -n -d test_genome_master_datastore_index.log

/vol_b/wq_maker_run/test_genome.maker.output


* -d The location of the MAKER datastore index log file.
* -n Do not print fasta sequence in footer

The final output from gff3_merge is **"test_genome.all.gff"**

In [27]:
!head test_genome.all.gff

##gff-version 3
Chr6	.	contig	1	100000	.	.	.	ID=Chr6;Name=Chr6
Chr6	maker	gene	33943	37124	.	+	.	ID=maker-Chr6-exonerate_protein2genome-gene-0.0;Name=maker-Chr6-exonerate_protein2genome-gene-0.0
Chr6	maker	mRNA	33943	37124	453	+	.	ID=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1;Parent=maker-Chr6-exonerate_protein2genome-gene-0.0;Name=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1;_AED=0.23;_eAED=0.23;_QI=0|0|0|1|0|0|5|0|150
Chr6	maker	exon	33943	33981	.	+	.	ID=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1:exon:0;Parent=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1
Chr6	maker	exon	34283	34350	.	+	.	ID=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1:exon:1;Parent=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1
Chr6	maker	exon	34749	34907	.	+	.	ID=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1:exon:2;Parent=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1
Chr6	maker	exon	35977	36055	.	+	.	ID=maker-Chr6-exonerate_protein2genome-gene-0.0-mRNA-1:exon:3

## Moving data from CyVerse Datastore using iCommands

iCommands is a collection of commands for Linux and Mac OS operating systems that are used in the iRODS system to interact with the CyVerse Data Store. Many commands are very similar to Unix utilities. For example, to list files and directories, in Linux you use ls, but in iCommands you use `ils`.

While iCommands are great for all transfers and for automating tasks via scripts, they are the best choice for large files (2-100 GB each) and for bulk file transfers (many small files). For a comparison of the different methods of uploading and downloading data items, see [Downloading and Uploading Data](https://pods.iplantcollaborative.org/wiki/display/DS/Downloading+and+Uploading+Data).

iCommands can be used by CyVerse account users to download files that have been shared by other users and to upload files to the Data Store, as well as [add metadata](https://wiki.cyverse.org/wiki/display/DS/Adding+Metadata+to+a+File+Using+iRODS+imeta+%28Metadata%29+Commands), change permissions, and more. Commonly used iCommands are listed below. Follow the instructions on [Setting Up iCommands](https://wiki.cyverse.org/wiki/display/DS/Setting+Up+iCommands) for how to download and configure iCommands for your operating system.

**A CyVerse account is not required to download a public data file via iCommands**. To see instructions just for public data download with iCommands, see the iCommands section on Downloading Data Files Without a User Account.